# GPT-SoVITS WebUI
Port made by [Nick088](https://linktr.ee/Nick088)

# Install

In [ ]:
from IPython.display import clear_output

# gptsovits
!git clone https://github.com/RVC-Boss/GPT-SoVITS.git
%cd /kaggle/working/GPT-SoVITS
!apt-get update && apt-get install -y --no-install-recommends tzdata ffmpeg libsox-dev parallel aria2 git git-lfs && git lfs install
!pip install -r requirements.txt
# filebrowser
!sudo curl -fsSL https://raw.githubusercontent.com/filebrowser/get/master/get.sh | sudo bash
!filebrowser config init
!filebrowser config set --auth.method=noauth
!filebrowser users add  "user" "" --perm.admin
!apt install psmisc # needed for killing later on the previously running processes

clear_output()
print("Installed!")

# Download pretrained models 下载预训练模型


In [ ]:
from IPython.display import clear_output

# gptsovits
%cd /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models
!git clone https://huggingface.co/lj1995/GPT-SoVITS
!mv /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /kaggle/working/GPT-SoVITS/GPT_SoVITS/pretrained_models/
!rm -rf GPT-SoVITS
# chinese asr
%cd /kaggle/working/GPT-SoVITS/tools/asr/models
!git clone https://www.modelscope.cn/damo/speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch.git
!git clone https://www.modelscope.cn/damo/speech_fsmn_vad_zh-cn-16k-common-pytorch.git
!git clone https://www.modelscope.cn/damo/punc_ct-transformer_zh-cn-common-vocab272727-pytorch.git
# uvr5
%cd /kaggle/working/GPT-SoVITS/tools/uvr5
!rm -rf uvr5_weights
!git clone https://huggingface.co/Delik/uvr5_weights

clear_output()
print("Downloaded!")

# launch WebUI 启动WebUI

The first run may take more time as it will download the g2pw model.

In [ ]:
from IPython.display import clear_output
import os 

%cd /kaggle/working/GPT-SoVITS/


#@markdown The type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one.
Tunnel = "LocalTunnel" #@param ["Ngrok", "LocalTunnel", "Horizon"]

#@markdown Also, you need to wait for the Local URL to appear, and only after that click on the Public URL which is above. Plus, you have to open UVR/Inference/Proofreading in the GPT-SoVITS UI before opening the Public URL.

#@markdown Use the following option **only if you chose Ngrok** as the Tunnel, **You need to be a paid ngrok member** as it needs more than 3 tunnels (there are 5 ports):

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.

ngrok_tunnel_authtoken = "" #@param {type:"string"}

#@markdown Use the following option **only if you chose Ngrok** as the Tunnel:

#@markdown You can get the Horizon ID here: https://hrzn.run/dashboard/ , login, on the 2nd step, it shows an `hrzn login YOUR_ID`, you need to copy that id

horizon_id = "" #@param {type:"string"}


%env is_share = False

if Tunnel == "Ngrok":
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_tunnel_authtoken)
  gptsovits_tunnel = ngrok.connect(9874, bind_tls=True)
  uvr_tunnel = ngrok.connect(9873, bind_tls=True)
  inference_tunnel = ngrok.connect(9872, bind_tls=True)
  proofreading_tunnel = ngrok.connect(9871, bind_tls=True)
  file_tunnel = ngrok.connect(9000, bind_tls=True)
  print("GPT-SoVITS Tunnel Public URL:", gptsovits_tunnel.public_url)
  print("UVR Tunnel Public URL:", uvr_tunnel.public_url)
  print("Inference Tunnel Public URL:", uvr_tunnel.public_url)
  print("Proofreading Tunnel Public URL:", proofreading_tunnel.public_url)
  print("File Tunnel Public URL:", file_tunnel.public_url)
elif Tunnel == "LocalTunnel":
  # install
  !npm install -g localtunnel
  import time
  import urllib
  # run localtunnel
  # gptsovits
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9874 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()

  print(f"GPT-SoVITS Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  # uvr
  with open('url2.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9873 >> url2.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url2.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  print(f"UVR Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  # inference
  with open('url3.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9872 >> url3.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url3.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  print(f"Inference Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  # proofreading
  with open('url4.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9871 >> url4.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url4.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  print(f"Proofreading Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  # file
  with open('url5.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 9000 >> url5.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url5.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  print(f"File Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

    
  print(f'LocalTunnels Password: {endpoint_ip}')
elif Tunnel == "Horizon":
  # install 
  !npm install -g @hrzn/cli
  # login
  !hrzn login $horizon_id
  os.system("hrzn tunnel http://localhost:9874 &")
  os.system("hrzn tunnel http://localhost:9873 &")
  os.system("hrzn tunnel http://localhost:9872 &")
  os.system("hrzn tunnel http://localhost:9871 &")
  os.system("hrzn tunnel http://localhost:9000 &")


# kills previously running processes
!fuser -k 9874/tcp
!fuser -k 9873/tcp
!fuser -k 9872/tcp
!fuser -k 9871/tcp
!fuser -k 9000/tcp


os.system("filebrowser -r /kaggle/working -p 9000 &")
!python webui.py